In [184]:
import pandas as pd
import numpy as np
import itertools
import os

from theano import function
import theano.tensor as tt

from utils.data import load_new_data, train_test_split
from utils.utils import init_weights, morphological_filter
from utils.cv import cv_threshold, cv_build_model, cv_fit_model
from utils.theano_pq import pq_theano_f, pq_theano, burnaev_metric
from utils.pass_quality import approx_pq

# theano_pq

In [22]:
y_t, y_p = tt.vector(name='y_t', dtype='float32'), tt.vector(name='y_p', dtype='float32')

In [24]:
f = function(inputs=[y_t, y_p], outputs=pq_theano_f(y_t, y_p))

In [28]:
f(y_train.astype('float32'), y_train.astype('float32'))

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [71]:
from keras.objectives import MSE
import theano
import theano.tensor as tt

In [156]:
conc_data['shield_morph'] = morphological_filter(conc_data['r.shield'].values, 3)
conc_data['con_morph'] = morphological_filter(conc_data['r.cor'].values, 3)

In [155]:
conc_data = load_new_data()

In [157]:
X_train, X_test, y_train, y_test = train_test_split(conc_data[['r.cor',
                                                               'con_morph',
                                                               'r.cor',
                                                               'shield_morph',                                    
                                                               #'r.pass',
                                                               'y']], 
                                                    test_size=0.2,
                                                    n_prev=19)

In [158]:
model = cv_build_model(X_train, [pq_theano])

In [160]:
model = cv_fit_model(model, X_train, y_train, nb_epoch=19, weights=True)

Epoch 16/19
Epoch 1/1
 6144/54996 [==>...........................] - ETA: 12s - loss: 0.1199 - pq_theano: 0.1176

KeyboardInterrupt: 

In [180]:
thres, sm = cv_threshold(X_test, y_test, model, [0])

In [181]:
print thres, sm

0.293022969891 0


In [182]:
y_train_pred = (model.predict(X_train) > thres).astype(int).flatten()

print approx_pq(y_train, morphological_filter(y_train_pred, sm))[:-1]
#print approx_pq(y_train, morphological_filter(y_train_lab, 0))[:-1]

(0.85150812064986725, 5872.0000000099999, 1024.0)


In [183]:
y_test_pred = (model.predict(X_test) > thres).astype(int).flatten()

print approx_pq(y_test, morphological_filter(y_test_pred, sm))[:-1]

#print approx_pq(y_test, morphological_filter(y_test_lab, 0))[:-1]

(0.88586956521808047, 1467.0000000099999, 189.0000000000001)


In [173]:
conc_data['res_c'] = np.concatenate([np.zeros(18), 
                                       morphological_filter(y_train_pred, sm).astype(int), 
                                       morphological_filter(y_test_pred, sm).astype(int)])

In [148]:
# approx_pq(conc_data['y'].astype(int), conc_data['r.pass'].astype(int))[:-1]

In [149]:
a, b, c = approx_pq(conc_data['y'].astype(int), conc_data['res_cor'].astype(int))[:-1]

(int(a * 100000) / 100000., int(b), int(c))

(0.89981, 7401, 824)

# shield + cor:
(0.88366, 7277, 958)
# only loop:
(0.86061, 6798, 1101);(0.86538, 6840, 1064);(0.87144, 6840, 1009);
# only pass:
(0.8879, 7077, 893);(0.88153, 7017, 943);(0.88531, 7048, 913);(0.88371, 7075, 931);(0.88371, 7075, 931);(0.87849, 7035, 973);
# only shield:
(0.87096, 7209, 1068); (0.8744, 7201, 1034);(0.8909, 7147, 875);(0.89305, 7190, 861);
(0.89437, 7248, 856); $ fantoms and propusk : 88
# only cor:
(0.8734, 7246, 1050); (0.8953, 7269, 850);(0.8988, 7320, 824);(0.8909, 7225, 884);(0.8857, 7136, 920);
(0.9038, 7287, 775);(0.8934, 7255, 865);
# stacking max:
(0.8919, 7274, 881)
# double stacking:
(0.89236408063544004, 7304, 881)

In [19]:
approx_pq(conc_data['y'].astype(int), conc_data['y'].astype(int))[:-1]

(1.0, 7873.0000000099999, 1e-13)

In [31]:
approx_pq(conc_data['y'].astype(int), conc_data['r.pass'].astype(int))[:-1]

(0.83167371436389081, 6097.0000000099999, 1234.0)

In [153]:
len(approx_pq(conc_data['y'].astype(int), conc_data['r.pass'].astype(int))[-1][0])

696

In [232]:
approx_pq_dbg(conc_data['y'].astype(int), conc_data['r.pass'].astype(int))[:-1]

(0.861114237478897, 7651, 1234)

In [233]:
bench_errors = approx_pq(conc_data['y'].astype(int), conc_data['r.pass'].astype(int))[-1][0]

In [234]:
n_fantoms_and_propusk = len([err[1] for err in bench_errors if err[1] == 1])

In [235]:
n_fantoms_and_propusk

165

In [127]:
bench_errors = approx_pq(conc_data['y'].astype(int), conc_data['res_cor'].astype(int))[-1][0]

In [128]:
n_fantoms_and_propusk = len([err[1] for err in bench_errors if err[1] == 1])

In [129]:
n_fantoms_and_propusk

101

In [151]:
approx_pq(conc_data['y'].astype(int)[20:], conc_data['res_cor'].astype(int).astype(int)[20:])[-1][-1]

207